# VacationPy

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
final_cities = "../WeatherPy/cities.csv"

final_cities_df = pd.read_csv(final_cities)
final_cities_df.drop(["Unnamed: 0"], axis=1, inplace=True)

final_cities_df.head()

,country,city,lat,lng,temp,humidity,cloudiness,wind speed,datetime
0,US,Jamestown,42.0970,-79.2353,-7.78,73,1,3.09,1613085479
1,ZA,Cape Town,-33.9258,18.4232,22.22,94,20,4.63,1613085100
2,GL,Upernavik,72.7868,-56.1549,-17.76,81,88,2.74,1613085482
3,NZ,Bluff,-46.6000,168.3333,14.44,72,42,4.02,1613085483
4,CN,Hailun,47.4500,126.9333,-7.17,96,32,3.82,1613085484


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Configure gmaps
gmaps.configure(api_key=g_key)

# Store lat and lng in locations
locations = final_cities_df[["lat","lng"]]

# Store humidity to set weight and make sure value is integer
humidity = final_cities_df["humidity"].astype(int)

In [4]:
# Plot heatmap
heatmap_fig = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=200, point_radius=3)
heatmap_fig = gmaps.figure(center=(20,5), zoom_level=2.2)

# Add heat layer
heatmap_fig.add_layer(heat_layer)

# Display
heatmap_fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows with null values.

In [5]:
# Drop rows not in the 70-80F degree range (21-27C) || not too humid (60+) || windy cities (+5mph) || cloudy cities (+0)
ideal_cities_df = final_cities_df.drop(final_cities_df[(final_cities_df["temp"] > 27) |
                                                       (final_cities_df["temp"] < 21) |
                                                       (final_cities_df["humidity"] > 60) |
                                                       (final_cities_df["wind speed"] > 5) |
                                                       (final_cities_df["cloudiness"] > 0)
                                                      ].index, inplace=False)

# Display
ideal_cities_df

,country,city,lat,lng,temp,humidity,cloudiness,wind speed,datetime
38,NZ,Whitianga,-36.8333,175.7000,21.08,45,0,2.59,1613085532
43,BF,Sapouy,11.5544,-1.7736,25.43,16,0,1.97,1613085539
107,BF,Diapaga,12.0708,1.7889,25.19,15,0,3.16,1613085626
186,MX,Leyva Solano,25.6667,-108.6417,23.89,54,0,2.24,1613085739
272,NZ,Ngunguru,-35.6167,174.5000,23.33,56,0,1.79,1613085854
287,CM,Tcholliré,8.4022,14.1698,21.30,20,0,0.39,1613085872
301,SD,Wad Medani,14.4012,33.5199,24.09,22,0,3.62,1613085890
316,OM,Salalah,17.0151,54.0924,21.00,56,0,2.06,1613085911
522,NG,Kumo,10.0447,11.2122,22.05,20,0,3.08,1613086180


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [29]:
# Create hotel_df and add column "hotel name"
hotel_df = ideal_cities_df
hotel_df["hotel name"] = ""

# Set search parameters
hotel_params = {
    "types": "lodging",
    "radius": 5000,
    "key": g_key
}

for index,row in hotel_df.iterrows():
    # Get lat, lng
    Lat = row["lat"]
    Lng = row["lng"]
    
    # change location each iteration while leaving original params in place
    hotel_params["location"] = f"{Lat},{Lng}"
    
    # Create query, make request, and store in json
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    hotel_response = requests.get(base_url, params=hotel_params)
    hotels = hotel_response.json()
    
    #Try to gather hotel names, but skip for those with no hotels in the set radius
    try:
        hotel_df.loc[index, "hotel name"] = hotels["results"][0]["name"]
        
    except (KeyError, IndexError):
        pass
    
# Display updated df
hotel_df

,country,city,lat,lng,temp,humidity,cloudiness,wind speed,datetime,hotel name
38,NZ,Whitianga,-36.8333,175.7000,21.08,45,0,2.59,1613085532,Beachside Resort Whitianga
43,BF,Sapouy,11.5544,-1.7736,25.43,16,0,1.97,1613085539,Auberge St Louis
107,BF,Diapaga,12.0708,1.7889,25.19,15,0,3.16,1613085626,
186,MX,Leyva Solano,25.6667,-108.6417,23.89,54,0,2.24,1613085739,The Big Boss House
272,NZ,Ngunguru,-35.6167,174.5000,23.33,56,0,1.79,1613085854,Pacific Rendezvous Motel Resort
287,CM,Tcholliré,8.4022,14.1698,21.30,20,0,0.39,1613085872,"Auberge ""l'amicale"""
301,SD,Wad Medani,14.4012,33.5199,24.09,22,0,3.62,1613085890,فندق يوغسلافيا
316,OM,Salalah,17.0151,54.0924,21.00,56,0,2.06,1613085911,HAMDAN PLAZA HOTEL SALALAH
522,NG,Kumo,10.0447,11.2122,22.05,20,0,3.08,1613086180,Ajegunle Lodge(corpers Lodge)


In [37]:
# Drop Diapaga, BF since there is no hotel nearby
hotel_df.drop([107], inplace=True)
hotel_df

,country,city,lat,lng,temp,humidity,cloudiness,wind speed,datetime,hotel name
38,NZ,Whitianga,-36.8333,175.7000,21.08,45,0,2.59,1613085532,Beachside Resort Whitianga
43,BF,Sapouy,11.5544,-1.7736,25.43,16,0,1.97,1613085539,Auberge St Louis
186,MX,Leyva Solano,25.6667,-108.6417,23.89,54,0,2.24,1613085739,The Big Boss House
272,NZ,Ngunguru,-35.6167,174.5000,23.33,56,0,1.79,1613085854,Pacific Rendezvous Motel Resort
287,CM,Tcholliré,8.4022,14.1698,21.30,20,0,0.39,1613085872,"Auberge ""l'amicale"""
301,SD,Wad Medani,14.4012,33.5199,24.09,22,0,3.62,1613085890,فندق يوغسلافيا
316,OM,Salalah,17.0151,54.0924,21.00,56,0,2.06,1613085911,HAMDAN PLAZA HOTEL SALALAH
522,NG,Kumo,10.0447,11.2122,22.05,20,0,3.08,1613086180,Ajegunle Lodge(corpers Lodge)


In [40]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{hotel name}</dd>
<dt>City</dt><dd>{city}</dd>
<dt>Country</dt><dd>{country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["lat", "lng"]]

In [45]:
# Add marker layer ontop of heat map
hotel_layer = gmaps.symbol_layer(
    locations, fill_color='rgba(0,150,0,0.4)',
    stroke_color='rgba(0,0,150,0.4)', scale=10,
    info_box_content=hotel_info
)

heatmap_fig = gmaps.figure(center=(20,5), zoom_level=2.2)
heatmap_fig.add_layer(heat_layer)
heatmap_fig.add_layer(hotel_layer)

# Display figure
heatmap_fig

Figure(layout=FigureLayout(height='420px'))